After finishing this tutorial's first notebook, you should now have around 380 MIDI files of guitar solo music. In this notebook, you will convert your MIDI files into text token representations and upload your new text dataset to the Hugging Face hub. Let's dive in!

## 2.1 Representing MIDI files as text tokens

Before uploading your dataset to the Hugging Face hub, you'll convert your MIDI files to text tokens. Doing this will allow you to use some state-of-the-art casual language models (GPT-2 for this tutorial).

There are many ways to do this, and this is a field in constant evolution. Let's use the text representations of our MIDI files as described in the "MMM: Exploring Conditional Multi-Track Music Generation with the Transformer" [paper](https://arxiv.org/abs/2008.06048). I highly encourage you to read this paper. It has a lot of details that we won't cover here, and it is not a difficult read. 

In this representation, you convert music events to text tokens such as:

*   **"PIECE_START" and "PIECE_END"**: Represent your piece's beginning and end.
*   **"TRACK_START" and "TRACK_END"**: Cover the duration of each track. 
*   **"BAR_START" and "BAR_END"**: Represent a musical bar's span.

Inside pieces, you have tracks. Inside tracks, you have bars. And inside bars, you have notes with pitches and durations, as shown in this diagram from the paper:

![picture](https://drive.google.com/uc?id=1lvUIwFH3ZPhvhy6LUN4vVS7ZvlfszQvC)

Here are some examples of the tokens used to represent musical events inside bars:
*   **"NOTE_ON=60"**: Represents pressing the note with pitch C4 (60 is the MIDI number for C4).
*   **"TIME_DELTA=2"**: Duration in pulses since the last event.
*   **"NOTE_OFF=60"**: This means stop pressing the note with pitch C4.

To illustrate this better, consider the following bar from Op. 50 No. 04 of Giuliani:

![picture](https://drive.google.com/uc?id=1x_napL4BM3EOuXiwtAKBj0pta6vAGlCY)

The tokenization process will represent this bar with the following tokens:
* PIECE_START 
* TIME_SIGNATURE=6_8 
* BPM=108
  * TRACK_START 
  * INST=0 
  * DENSITY=4 
  * BAR_START 
    * NOTE_ON=74 TIME_DELTA=2.0 NOTE_OFF=74 NOTE_ON=74 NOTE_ON=55 TIME_DELTA=2.0 NOTE_OFF=55 NOTE_ON=59 TIME_DELTA=2.0 NOTE_OFF=59 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=74 NOTE_ON=71 NOTE_OFF=62 NOTE_ON=55 TIME_DELTA=2.0 NOTE_OFF=55 NOTE_ON=59 TIME_DELTA=2.0 NOTE_OFF=71 NOTE_OFF=59 
  * BAR_END 

Notice the use of the tokens **TIME_SIGNATURE=** and **BPM=** to denote the meter and tempo of the bar, respectively. The token **INST=** represents the instrument in MIDI notation, and **DENSITY=** indicates the number of notes inside a bar: More density means more notes. I always use INST=0 (piano) since we are using only Guitar. You'll change this later when creating the sound output of the model.

Converting the MIDI files into this token representation is not a trivial task, but fortunately [Dr. Tristan Behrens](https://www.linkedin.com/in/dr-tristan-behrens-734967a2/) open-sourced an excellent code for doing this. I  tweaked the code a bit so it fits our needs for the Mutopia Project.

So to convert your MIDI files to text tokens, please do the following:

*   Go to the repo [MMM Tokenization](https://github.com/juancopi81/MMM_Tokenization)
*   Clone the repo to your local computer
*   Add the MIDI files to the folder `/datasets/custom_midi_dataset/`.
*   Create the MMMTrack datasets with `python create_dataset_mmm.py`.

After doing that, you should have two new text files inside the folder `/datasets/custom_dataset_mmmtrack/`:

*   `token_sequences_train.txt`
*   `token_sequences_valid.txt`

Feel free also to use the files that are already in the repo. You'll use these files to upload your dataset to Hugging Face.

## 2.2 Uploading your dataset to Hugging Face

If you are working in Colab, for this part of this notebook, you need to upload the token_sequences_train.txt and token_sequences_valid.txt files in the folder `/content/mutopia_guitar_dataset_files/`. You also need to have a Hugging Face account for uploading your model. With all that set, you can follow the next instructions; I won't go into many details here since most of them are self-explanatory.

Install the Transformers and Datasets libraries.

In [ ]:
!pip install datasets transformers[sentencepiece]
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 2.5 MB/s 
     |████████████████████████████████| 4.4 MB 53.1 MB/s 
     |████████████████████████████████| 212 kB 68.4 MB/s 
     |████████████████████████████████| 140 kB 52.8 MB/s 
     |████████████████████████████████| 1.1 MB 49.6 MB/s 
     |████████████████████████████████| 101 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 43.6 MB/s 
     |████████████████████████████████| 127 kB 49.7 MB/s 
     |████████████████████████████████| 144 kB 53.5 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 58.7 MB/s 
     |████████████████████████████████| 6.6 MB 46.1 MB/s 
     |████████████████████████████████| 1.2 MB 53.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3

Setup git

In [ ]:
!git config --global user.email "me@hotmail.com"
!git config --global user.name "My name"

Log in to the Hugging Face Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Create repository of the dataset in 🤗 Hub.

In [ ]:
from huggingface_hub import create_repo

repo_url = create_repo(name="mutopia_guitar_dataset", repo_type="dataset")
repo_url

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:94: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.10. Pass `repo_id` instead.
  FutureWarning,


'https://huggingface.co/datasets/juancopi81/mutopia_guitar_dataset'

Clone the repository from the Hub to local machine and copy dataset files into it

In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir="mutopia_guitar_dataset", clone_from=repo_url)

Cloning https://huggingface.co/datasets/juancopi81/mutopia_guitar_dataset into local empty directory.


In [ ]:
!cp /content/mutopia_guitar_dataset_files/* /content/mutopia_guitar_dataset/

Include the `.txt` format in the list of the .gitattributes file so Git LFS tracks these files.

In [ ]:
repo.lfs_track("*.txt")

Push the dataset to the Hub:

In [ ]:
repo.push_to_hub()

Upload file train.txt:   0%|          | 3.34k/94.9M [00:00<?, ?B/s]

Upload file test.txt:   0%|          | 3.34k/882k [00:00<?, ?B/s]

To https://huggingface.co/datasets/juancopi81/mutopia_guitar_dataset
   48c662e..0a3f9a8  main -> main



'https://huggingface.co/datasets/juancopi81/mutopia_guitar_dataset/commit/0a3f9a8575c56bfa63dc94a9a869294e47b94043'

Load the dataset from the hub.

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("juancopi81/mutopia_guitar_dataset")
raw_datasets

Using custom data configuration juancopi81--mutopia_guitar_dataset-7b7b35ea85ad98f4
Reusing dataset text (/root/.cache/huggingface/datasets/juancopi81___text/juancopi81--mutopia_guitar_dataset-7b7b35ea85ad98f4/0.0.0/acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7325
    })
    test: Dataset({
        features: ['text'],
        num_rows: 74
    })
})

Congratulation!! You now have your dataset in the Hugging Face hub. In the next notebook, you'll create your tokenizer for the model you train. See you there.